In [35]:
import pandas as pd
import numpy as np
ARIMA_forecast = pd.read_pickle('forecast.pkl')

In [36]:
############ Evaluation #############

def MSE(df, method='ARIMA'):
    temp = df.copy()
    error = temp['grid-loss']-temp[method]
    squared= error**2
    mse = squared.mean()
    return mse
def MAE(df, method='ARIMA'):
    temp = df.copy()
    error = temp['grid-loss']-temp[method]
    absolute = abs(error)
    mae = absolute.mean()
    return mae

def MAPE(df, method='ARIMA'):
    temp = df.copy()
    mae = MAE(temp, method)
    return mae / temp['grid-loss'].mean()

MSE(ARIMA_forecast, 'ARIMA')
    


19.04176484471903